<a href="https://colab.research.google.com/github/TF2113/mobility-aid/blob/master/ml/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a Model using YOLO Ultralytics

Steps to train a model on an image dataset for use with a Raspberry Pi 5. Using a lightweight version of the YOLO algorithm to efficiently run inference on a low power device.

## Gain access to a GPU for training purposes.

Ensure runtime settings are set to use T4-GPU hardware acceleration and run this code to verify GPU connection.

In [ ]:
!nvidia-smi

## Install Ultralytics

Run this code to install the Ultralytics library to this runtime session. This will be used to train the model.

In [ ]:
!pip install ultralytics

## File Handling

Upload the augmented data.zip file outputted from augmentation.ipynb to the files section on the sidebar.

Run this code on the output of augmentation.ipynb to split the dataset into training and validation sections.

In [ ]:
import os
import shutil

!unzip -q /content/data.zip -d /content/training_data

folder_pairs = [
    ('/content/training_data/images', '/content/training_data/val/images'),
    ('/content/training_data/labels', '/content/training_data/val/labels'),
    ('/content/training_data/aug_images', '/content/training_data/train/images'),
    ('/content/training_data/aug_labels', '/content/training_data/train/labels'),
]

for src_dir, dest_dir in folder_pairs:
    os.makedirs(dest_dir, exist_ok=True)
    for filename in os.listdir(src_dir):
        src_file = os.path.join(src_dir, filename)
        dest_file = os.path.join(dest_dir, filename)
        if os.path.isfile(src_file):
            shutil.move(src_file, dest_file)

Once the images have been split into training data and validation data, a simple data.yaml file can be used to provide the structure of the data for training the model. Once the yaml file has been uploaded to the root directory, the training can be run with the code below.

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11n.pt epochs=150 imgsz=640

Now the model has been trained, it will be converted to NCNN format for portability and allow running inference with C++ on Raspberry Pi 4 or other low-power Linux based devices.

In [ ]:
from ultralytics import YOLO

model = YOLO("./runs/detect/train2/weights/best.pt")

model.export(format="ncnn")  # creates '/yolo11n_ncnn_model'


The output "model.param" and "model.bin" files can be transferred to the project directory for use with inference.